# Visualize tracking dataset

Load a sample from `MedicanesTrackDataset` and overlay the cyclone centre on the video tile.

Set `csv_path` and `data_root` to point to your local data.

In [ ]:
import os, re
import numpy as np
import torch
from torch.utils.data import DataLoader
from dataset.tracking_dataset import MedicanesTrackDataset
from dataset.build_dataset import get_cyclone_center_pixel
from view_test_tiles import display_video_clip
from PIL import Image, ImageDraw


In [ ]:
# Paths to the annotation CSV and tile frames
csv_path = 'PATH/TO/tracking_annotations.csv'
data_root = 'PATH/TO/tiles'

# Create dataset and loader
track_ds = MedicanesTrackDataset(csv_path, data_root=data_root)
loader = DataLoader(track_ds, batch_size=1, shuffle=True)


In [ ]:
def extract_offsets(folder_path: str):
    """Parse tile top-left corner (x, y) from folder name."""
    base = os.path.basename(folder_path.rstrip(os.sep))
    nums = [int(n) for n in re.findall(r'\d+', base)]
    if len(nums) >= 2:
        return nums[-2], nums[-1]
    return 0, 0


In [ ]:
# Grab a single example
video, coords, folder = track_ds[0]

# Undo normalization to [0, 1]
mean = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1,1)
std = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1,1)
video = (video * std + mean).clamp(0, 1)

# Convert to numpy [T, H, W, C]
video_np = video.permute(1, 2, 3, 0).numpy()

# Convert geographic coords to pixel position in the big frame
lon, lat = coords.tolist()
px, py = get_cyclone_center_pixel(lat, lon)

# Extract tile offset from folder path and compute position in the tile
x_off, y_off = extract_offsets(folder)
px_tile = px - x_off
py_tile = py - y_off

# Draw the cyclone centre on each frame
frames = []
for frame in video_np:
    img = (frame * 255).astype(np.uint8)
    pil = Image.fromarray(img)
    draw = ImageDraw.Draw(pil)
    r = 4
    draw.ellipse((px_tile - r, py_tile - r, px_tile + r, py_tile + r), fill=(255, 0, 0))
    frames.append(np.asarray(pil) / 255.0)

# Display the clip
display_video_clip(frames)
